In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import gensim
import sys
import os
from string import punctuation
from gensim.utils import deaccent
import utils
from multiprocessing import Pool,cpu_count
from itertools import chain
from pathlib import Path
import pickle

2018-12-29 02:32:58,053 : INFO : 'pattern' package not found; tag filters are not available for English


In [2]:
class BillionWordCorpus(object):
    def __init__(self, workers=-1, override=True):
        self.override = override
        self.workers = workers if workers != -1 else cpu_count()
        self.dirname = Path("../data/clean_corpus/spanish_billion_words")
        self.prepare()
    
    def prepare(self):
        files = [file for file in self.dirname.iterdir() if file.is_file() and "_preprocessed" not in str(file)]
        logging.getLogger(__name__).info(f"starting preprocessing with {self.workers} cores")
        
        if not self.override:
            files = [file for file in files if not os.path.exists(f"{str(file)}_preprocessed")]
        
        logging.getLogger(__name__).info(f"files to preprocess: {len(files)}")
        
        with Pool(self.workers, maxtasksperchild=2) as pool:
            pool.map(self.aux, files)
        
        logging.getLogger(__name__).info(f"finished preprocessing")
    
    @staticmethod
    def aux(fname):
        alll = []
        logging.getLogger(__name__).info(f"starting {fname}")
        fname_preprocessed = f"{fname}_preprocessed" 

        with open(fname) as open_file:
            for line in open_file:
                line = "".join([x for x in line if x not in utils.non_words])
                l = map(lambda x: x.strip().lower(), deaccent(line).split())
                l = [*filter(lambda x: x not in utils.stopwords, l)]
                alll.append([l])

        with open(fname_preprocessed, 'wb') as f:
            pickle.dump(alll, f, protocol=pickle.HIGHEST_PROTOCOL)

        logging.getLogger(__name__).info(f"finished {fname}")
    
    def __iter__(self):
        files = [file for file in self.dirname.iterdir() if file.is_file() and "_preprocessed" in str(file)]
        for f in files:
            with open(f, 'rb') as open_file:
                my_list = pickle.load(open_file)
                for line in my_list:
                    yield line[0]

In [4]:
sentences = BillionWordCorpus(override=False)

2018-12-29 02:32:58,258 : INFO : starting preprocessing with 4 cores
2018-12-29 02:32:58,262 : INFO : files to preprocess: 0
2018-12-29 02:32:58,392 : INFO : finished preprocessing


In [5]:
override = True
if not os.path.exists("model_bwc") or override:
#     bigram_transformer = gensim.models.Phrases(sentences)
    model_bwc = gensim.models.Word2Vec(sentences, min_count=40, size=300, workers=2, window=5)
    model_bwc.save("model_bwc")
else:
    model_bwc = gensim.models.Word2Vec.load("model_bwc")

2018-12-29 02:32:58,417 : INFO : collecting all words and their counts
2018-12-29 02:33:01,766 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-12-29 02:33:01,833 : INFO : PROGRESS: at sentence #10000, processed 257479 words, keeping 16263 word types
2018-12-29 02:33:01,875 : INFO : PROGRESS: at sentence #20000, processed 503572 words, keeping 23575 word types
2018-12-29 02:33:01,920 : INFO : PROGRESS: at sentence #30000, processed 725577 words, keeping 28470 word types
2018-12-29 02:33:01,970 : INFO : PROGRESS: at sentence #40000, processed 948716 words, keeping 31978 word types
2018-12-29 02:33:02,019 : INFO : PROGRESS: at sentence #50000, processed 1173243 words, keeping 35940 word types
2018-12-29 02:33:02,070 : INFO : PROGRESS: at sentence #60000, processed 1414128 words, keeping 39181 word types
2018-12-29 02:33:02,128 : INFO : PROGRESS: at sentence #70000, processed 1660113 words, keeping 42344 word types
2018-12-29 02:33:02,177 : INFO : PROGRESS: 

2018-12-29 02:33:10,462 : INFO : PROGRESS: at sentence #720000, processed 16528145 words, keeping 116874 word types
2018-12-29 02:33:10,513 : INFO : PROGRESS: at sentence #730000, processed 16760456 words, keeping 117498 word types
2018-12-29 02:33:10,562 : INFO : PROGRESS: at sentence #740000, processed 16985010 words, keeping 118077 word types
2018-12-29 02:33:10,617 : INFO : PROGRESS: at sentence #750000, processed 17222674 words, keeping 118556 word types
2018-12-29 02:33:10,667 : INFO : PROGRESS: at sentence #760000, processed 17459708 words, keeping 119223 word types
2018-12-29 02:33:10,730 : INFO : PROGRESS: at sentence #770000, processed 17735350 words, keeping 119729 word types
2018-12-29 02:33:10,778 : INFO : PROGRESS: at sentence #780000, processed 17958544 words, keeping 120554 word types
2018-12-29 02:33:10,828 : INFO : PROGRESS: at sentence #790000, processed 18190187 words, keeping 121053 word types
2018-12-29 02:33:10,880 : INFO : PROGRESS: at sentence #800000, processe

2018-12-29 02:33:16,282 : INFO : PROGRESS: at sentence #1430000, processed 29430348 words, keeping 316031 word types
2018-12-29 02:33:19,073 : INFO : PROGRESS: at sentence #1440000, processed 29618428 words, keeping 317654 word types
2018-12-29 02:33:19,116 : INFO : PROGRESS: at sentence #1450000, processed 29827943 words, keeping 318018 word types
2018-12-29 02:33:19,169 : INFO : PROGRESS: at sentence #1460000, processed 30051098 words, keeping 318520 word types
2018-12-29 02:33:19,218 : INFO : PROGRESS: at sentence #1470000, processed 30280377 words, keeping 319020 word types
2018-12-29 02:33:19,271 : INFO : PROGRESS: at sentence #1480000, processed 30518767 words, keeping 319455 word types
2018-12-29 02:33:19,325 : INFO : PROGRESS: at sentence #1490000, processed 30767985 words, keeping 320015 word types
2018-12-29 02:33:19,374 : INFO : PROGRESS: at sentence #1500000, processed 30992228 words, keeping 320328 word types
2018-12-29 02:33:19,425 : INFO : PROGRESS: at sentence #1510000,

2018-12-29 02:33:25,327 : INFO : PROGRESS: at sentence #2140000, processed 44417697 words, keeping 371698 word types
2018-12-29 02:33:25,380 : INFO : PROGRESS: at sentence #2150000, processed 44584081 words, keeping 372994 word types
2018-12-29 02:33:25,436 : INFO : PROGRESS: at sentence #2160000, processed 44750489 words, keeping 374296 word types
2018-12-29 02:33:25,491 : INFO : PROGRESS: at sentence #2170000, processed 44918027 words, keeping 375633 word types
2018-12-29 02:33:25,546 : INFO : PROGRESS: at sentence #2180000, processed 45081805 words, keeping 376914 word types
2018-12-29 02:33:25,598 : INFO : PROGRESS: at sentence #2190000, processed 45248223 words, keeping 378196 word types
2018-12-29 02:33:25,653 : INFO : PROGRESS: at sentence #2200000, processed 45414169 words, keeping 379420 word types
2018-12-29 02:33:25,713 : INFO : PROGRESS: at sentence #2210000, processed 45578727 words, keeping 380669 word types
2018-12-29 02:33:25,767 : INFO : PROGRESS: at sentence #2220000,

2018-12-29 02:33:32,009 : INFO : PROGRESS: at sentence #2850000, processed 56226487 words, keeping 447124 word types
2018-12-29 02:33:32,051 : INFO : PROGRESS: at sentence #2860000, processed 56373832 words, keeping 447588 word types
2018-12-29 02:33:32,096 : INFO : PROGRESS: at sentence #2870000, processed 56521955 words, keeping 448040 word types
2018-12-29 02:33:32,142 : INFO : PROGRESS: at sentence #2880000, processed 56671109 words, keeping 448505 word types
2018-12-29 02:33:32,183 : INFO : PROGRESS: at sentence #2890000, processed 56814361 words, keeping 449004 word types
2018-12-29 02:33:32,226 : INFO : PROGRESS: at sentence #2900000, processed 56960285 words, keeping 449459 word types
2018-12-29 02:33:32,269 : INFO : PROGRESS: at sentence #2910000, processed 57107162 words, keeping 449929 word types
2018-12-29 02:33:32,313 : INFO : PROGRESS: at sentence #2920000, processed 57257322 words, keeping 450833 word types
2018-12-29 02:33:32,366 : INFO : PROGRESS: at sentence #2930000,

...

2018-12-29 05:05:17,569 : INFO : EPOCH 5 - PROGRESS: at 81.78% examples, 390907 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:05:18,587 : INFO : EPOCH 5 - PROGRESS: at 81.84% examples, 390928 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:05:19,593 : INFO : EPOCH 5 - PROGRESS: at 81.90% examples, 390953 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:05:20,612 : INFO : EPOCH 5 - PROGRESS: at 81.96% examples, 390968 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:05:21,867 : INFO : EPOCH 5 - PROGRESS: at 82.00% examples, 390863 words/s, in_qsize 2, out_qsize 0
2018-12-29 05:05:24,501 : INFO : EPOCH 5 - PROGRESS: at 82.01% examples, 390191 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:05:25,519 : INFO : EPOCH 5 - PROGRESS: at 82.09% examples, 390261 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:05:26,534 : INFO : EPOCH 5 - PROGRESS: at 82.16% examples, 390345 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:05:27,538 : INFO : EPOCH 5 - PROGRESS: at 82.23% examples, 390358 words/s, in_qsiz

2018-12-29 05:06:39,741 : INFO : EPOCH 5 - PROGRESS: at 86.54% examples, 390931 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:06:40,763 : INFO : EPOCH 5 - PROGRESS: at 86.60% examples, 390959 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:06:41,786 : INFO : EPOCH 5 - PROGRESS: at 86.66% examples, 390981 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:06:42,807 : INFO : EPOCH 5 - PROGRESS: at 86.73% examples, 391010 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:06:43,808 : INFO : EPOCH 5 - PROGRESS: at 86.79% examples, 391038 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:06:44,813 : INFO : EPOCH 5 - PROGRESS: at 86.85% examples, 391066 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:06:47,597 : INFO : EPOCH 5 - PROGRESS: at 86.88% examples, 390490 words/s, in_qsize 2, out_qsize 1
2018-12-29 05:06:48,607 : INFO : EPOCH 5 - PROGRESS: at 86.95% examples, 390534 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:06:49,610 : INFO : EPOCH 5 - PROGRESS: at 87.03% examples, 390567 words/s, in_qsiz

2018-12-29 05:08:02,460 : INFO : EPOCH 5 - PROGRESS: at 90.76% examples, 390747 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:08:03,463 : INFO : EPOCH 5 - PROGRESS: at 90.82% examples, 390775 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:08:04,468 : INFO : EPOCH 5 - PROGRESS: at 90.87% examples, 390780 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:08:05,474 : INFO : EPOCH 5 - PROGRESS: at 90.93% examples, 390802 words/s, in_qsize 2, out_qsize 1
2018-12-29 05:08:06,486 : INFO : EPOCH 5 - PROGRESS: at 90.99% examples, 390828 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:08:09,966 : INFO : EPOCH 5 - PROGRESS: at 91.02% examples, 390149 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:08:10,981 : INFO : EPOCH 5 - PROGRESS: at 91.08% examples, 390216 words/s, in_qsize 4, out_qsize 0
2018-12-29 05:08:12,000 : INFO : EPOCH 5 - PROGRESS: at 91.13% examples, 390251 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:08:13,012 : INFO : EPOCH 5 - PROGRESS: at 91.18% examples, 390319 words/s, in_qsiz

2018-12-29 05:09:25,823 : INFO : EPOCH 5 - PROGRESS: at 95.26% examples, 390436 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:09:26,829 : INFO : EPOCH 5 - PROGRESS: at 95.31% examples, 390500 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:09:27,871 : INFO : EPOCH 5 - PROGRESS: at 95.37% examples, 390565 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:09:28,876 : INFO : EPOCH 5 - PROGRESS: at 95.42% examples, 390638 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:09:29,890 : INFO : EPOCH 5 - PROGRESS: at 95.47% examples, 390673 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:09:30,905 : INFO : EPOCH 5 - PROGRESS: at 95.53% examples, 390755 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:09:31,906 : INFO : EPOCH 5 - PROGRESS: at 95.58% examples, 390831 words/s, in_qsize 4, out_qsize 0
2018-12-29 05:09:32,928 : INFO : EPOCH 5 - PROGRESS: at 95.64% examples, 390905 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:09:33,947 : INFO : EPOCH 5 - PROGRESS: at 95.69% examples, 390940 words/s, in_qsiz

2018-12-29 05:10:48,394 : INFO : EPOCH 5 - PROGRESS: at 99.50% examples, 390914 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:10:49,405 : INFO : EPOCH 5 - PROGRESS: at 99.56% examples, 390928 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:10:50,415 : INFO : EPOCH 5 - PROGRESS: at 99.62% examples, 390952 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:10:51,417 : INFO : EPOCH 5 - PROGRESS: at 99.68% examples, 390973 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:10:52,436 : INFO : EPOCH 5 - PROGRESS: at 99.73% examples, 390980 words/s, in_qsize 4, out_qsize 0
2018-12-29 05:10:53,448 : INFO : EPOCH 5 - PROGRESS: at 99.78% examples, 390973 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:10:54,456 : INFO : EPOCH 5 - PROGRESS: at 99.83% examples, 390967 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:10:55,464 : INFO : EPOCH 5 - PROGRESS: at 99.88% examples, 390955 words/s, in_qsize 3, out_qsize 0
2018-12-29 05:10:56,483 : INFO : EPOCH 5 - PROGRESS: at 99.94% examples, 390947 words/s, in_qsiz

In [6]:
model_bwc.wv.most_similar(positive=["rey","mujer"],negative=["hombre"],topn=1)

2018-12-29 05:10:59,509 : INFO : precomputing L2-norms of word weight vectors


[('reina', 0.5281879901885986)]

In [7]:
model_bwc.wv.most_similar(positive=["roma","francia"],negative=["italia"], topn=1)

[('paris', 0.7016277313232422)]

In [8]:
model_bwc.wv.most_similar(positive=["buenos","caballo"],negative=["bueno"], topn=1)

[('briosos', 0.45781493186950684)]

In [9]:
model_bwc.wv.most_similar(positive=["lindo","bueno"],negative=["feo"], topn=1)

[('estupendo', 0.5906842350959778)]